<a href="https://colab.research.google.com/github/aderdouri/PINNs/blob/master/Tutorials/my_inverse_bs_pinn.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Burgers equation:
## Problem setup

We will solve a Burgers' equation:

$$
\frac{\partial u}{\partial t} + u \frac{\partial u}{\partial x} = \nu \frac{\partial^2 u}{\partial x^2}, \quad x \in [-1, 1], \quad t \in [0, 1]
$$

with the Dirichlet boundary conditions and initial conditions:

$$
u(-1, t) = u(1, t) = 0, \quad u(x, 0) = -\sin(\pi x).
$$

The reference solution is [Here](https://github.com/lululxvi/deepxde/blob/master/examples/dataset/Burgers.npz)


In [2]:
import requests
import scipy.io
import torch
import io

In [3]:
# Step 1: GitHub raw URL to the .mat file
url = "https://github.com/maziarraissi/PINNs/raw/master/appendix/Data/burgers_shock.mat"

# Step 2: Download the file content
response = requests.get(url)
response.raise_for_status()  # Ensure the request was successful

# Step 3: Load the .mat file into memory
mat_data = scipy.io.loadmat(io.BytesIO(response.content))

# Step 4: Inspect the keys in the .mat file
print("Keys in the .mat file:", mat_data.keys())

# Step 5: Extract specific variables and convert to PyTorch tensors
usol = torch.tensor(mat_data['usol'], dtype=torch.float32)  # Solution variable
#x = torch.tensor(mat_data['x'], dtype=torch.float32)        # Spatial grid
#t = torch.tensor(mat_data['t'], dtype=torch.float32)        # Time grid

# Step 6: Print shapes of the loaded tensors
print(f"Shape of u(x, t): {usol.shape}")
#print(f"Shape of x: {x.shape}")
#print(f"Shape of t: {t.shape}")

Keys in the .mat file: dict_keys(['__header__', '__version__', '__globals__', 'x', 't', 'usol'])
Shape of u(x, t): torch.Size([256, 100])


## Implementation

In [4]:
import torch
import torch.nn as nn
import torch.optim as optim
import numpy as np
from scipy.io import loadmat
from scipy.interpolate import griddata
import matplotlib.pyplot as plt

In [7]:
import sys

import torch
import torch.nn as nn
import torch.optim as optim
import numpy as np
import matplotlib.pyplot as plt
import scipy.io
from scipy.interpolate import griddata
from mpl_toolkits.axes_grid1 import make_axes_locatable
import matplotlib.gridspec as gridspec
import time

np.random.seed(1234)
torch.manual_seed(1234)

class PhysicsInformedNN(nn.Module):
    def __init__(self, X, u, layers, lb, ub):
        super(PhysicsInformedNN, self).__init__()

        self.lb = lb
        self.ub = ub

        self.x = torch.tensor(X[:,0:1], dtype=torch.float32, requires_grad=True)
        self.t = torch.tensor(X[:,1:2], dtype=torch.float32, requires_grad=True)
        self.u = torch.tensor(u, dtype=torch.float32)

        self.layers = layers

        # Initialize NNs
        self.model = self.build_model(layers)

        # Initialize parameters
        self.lambda_1 = nn.Parameter(torch.tensor([0.0], dtype=torch.float32))
        self.lambda_2 = nn.Parameter(torch.tensor([-6.0], dtype=torch.float32))

        self.optimizer = optim.Adam(self.parameters())

    def build_model(self, layers):
        model = nn.Sequential()
        num_layers = len(layers)
        for i in range(num_layers - 1):
            model.add_module(f"layer_{i}", nn.Linear(layers[i], layers[i+1]))
            if i < num_layers - 2:
                model.add_module(f"tanh_{i}", nn.Tanh())
        return model

    def forward(self, x, t):
        X = torch.cat([x, t], dim=1)
        return self.model(X)

    def net_f(self, x, t):
        x.requires_grad = True
        t.requires_grad = True
        lambda_1 = self.lambda_1
        lambda_2 = torch.exp(self.lambda_2)
        u = self.forward(x, t)
        u_t = torch.autograd.grad(u, t, grad_outputs=torch.ones_like(u), create_graph=True)[0]
        u_x = torch.autograd.grad(u, x, grad_outputs=torch.ones_like(u), create_graph=True)[0]
        u_xx = torch.autograd.grad(u_x, x, grad_outputs=torch.ones_like(u_x), create_graph=True)[0]
        f = u_t + lambda_1 * u * u_x - lambda_2 * u_xx
        return f

    def loss_function(self, x, t, u):
        u_pred = self.forward(x, t)
        f_pred = self.net_f(x, t)
        loss = torch.mean((u - u_pred)**2) + torch.mean(f_pred**2)
        return loss

    def train_model(self, nIter):
        start_time = time.time()
        for it in range(nIter):
            self.optimizer.zero_grad()
            loss = self.loss_function(self.x, self.t, self.u)
            loss.backward()
            self.optimizer.step()

            if it % 100 == 0:
                elapsed = time.time() - start_time
                lambda_1_value = self.lambda_1.item()
                lambda_2_value = torch.exp(self.lambda_2).item()
                print(f'It: {it}, Loss: {loss.item():.3e}, Lambda_1: {lambda_1_value:.3f}, Lambda_2: {lambda_2_value:.6f}, Time: {elapsed:.2f}')
                start_time = time.time()

        # LBFGS optimizer for fine-tuning
        optimizer_lbfgs = torch.optim.LBFGS(model.parameters(),
                                            max_iter=50000,
                                            tolerance_grad=1e-7,
                                            tolerance_change=1e-9)

        lbfgs_iter = 0
        def closure():
            nonlocal lbfgs_iter
            optimizer_lbfgs.zero_grad()
            loss = self.loss_function(self.x, self.t, self.u)
            loss.backward()

            # Log progress
            if lbfgs_iter % 100 == 0:
                lambda_1_value = self.lambda_1.item()
                lambda_2_value = torch.exp(self.lambda_2).item()
                print(f"LBFGS Iteration {lbfgs_iter}, Loss: {loss.item():.6f}, lambda_1: {lambda_1_value:.6f}, lambda_2: {lambda_2_value:.6f}")
            lbfgs_iter += 1

            return loss

        print("Starting LBFGS optimization...")
        optimizer_lbfgs.step(closure)

    def predict(self, X_star):
        x_star = torch.tensor(X_star[:,0:1], dtype=torch.float32)
        t_star = torch.tensor(X_star[:,1:2], dtype=torch.float32)
        u_star = self.forward(x_star, t_star).detach().numpy()
        f_star = self.net_f(x_star, t_star).detach().numpy()
        return u_star, f_star

In [8]:
nu = 0.01/np.pi

N_u = 2000
layers = [2, 20, 20, 20, 20, 20, 20, 20, 20, 1]

data = mat_data

t = data['t'].flatten()[:,None]
x = data['x'].flatten()[:,None]
Exact = np.real(data['usol']).T

X, T = np.meshgrid(x,t)

X_star = np.hstack((X.flatten()[:,None], T.flatten()[:,None]))
u_star = Exact.flatten()[:,None]

# Domain bounds
lb = X_star.min(0)
ub = X_star.max(0)

######################################################################
######################## Noiseless Data ##############################
######################################################################
noise = 0.0

idx = np.random.choice(X_star.shape[0], N_u, replace=False)
X_u_train = X_star[idx,:]
u_train = u_star[idx,:]

model = PhysicsInformedNN(X_u_train, u_train, layers, lb, ub)
model.train_model(10000)

u_pred, f_pred = model.predict(X_star)

error_u = np.linalg.norm(u_star-u_pred,2)/np.linalg.norm(u_star,2)

U_pred = griddata(X_star, u_pred.flatten(), (X, T), method='cubic')

lambda_1_value = model.lambda_1.item()
lambda_2_value = torch.exp(model.lambda_2).item()

error_lambda_1 = np.abs(lambda_1_value - 1.0)*100
error_lambda_2 = np.abs(lambda_2_value - nu)/nu * 100

print('Error u: %e' % (error_u))
print('Error l1: %.5f%%' % (error_lambda_1))
print('Error l2: %.5f%%' % (error_lambda_2))

######################################################################
########################### Noisy Data ###############################
######################################################################
noise = 0.01
u_train = u_train + noise*np.std(u_train)*np.random.randn(u_train.shape[0], u_train.shape[1])

model = PhysicsInformedNN(X_u_train, u_train, layers, lb, ub)
model.train_model(10000)

u_pred, f_pred = model.predict(X_star)

lambda_1_value_noisy = model.lambda_1.item()
lambda_2_value_noisy = torch.exp(model.lambda_2).item()

error_lambda_1_noisy = np.abs(lambda_1_value_noisy - 1.0)*100
error_lambda_2_noisy = np.abs(lambda_2_value_noisy - nu)/nu * 100

print('Error lambda_1: %f%%' % (error_lambda_1_noisy))
print('Error lambda_2: %f%%' % (error_lambda_2_noisy))

It: 0, Loss: 5.697e-01, Lambda_1: 0.001, Lambda_2: 0.002478, Time: 0.08
It: 100, Loss: 1.089e-01, Lambda_1: -0.014, Lambda_2: 0.002609, Time: 2.89
It: 200, Loss: 3.343e-02, Lambda_1: 0.056, Lambda_2: 0.002298, Time: 3.51
It: 300, Loss: 2.855e-02, Lambda_1: 0.088, Lambda_2: 0.002308, Time: 3.56
It: 400, Loss: 2.509e-02, Lambda_1: 0.136, Lambda_2: 0.002290, Time: 2.84
It: 500, Loss: 2.320e-02, Lambda_1: 0.173, Lambda_2: 0.002333, Time: 2.88
It: 600, Loss: 2.245e-02, Lambda_1: 0.195, Lambda_2: 0.002453, Time: 2.98
It: 700, Loss: 2.148e-02, Lambda_1: 0.221, Lambda_2: 0.002597, Time: 4.12
It: 800, Loss: 1.995e-02, Lambda_1: 0.257, Lambda_2: 0.002750, Time: 2.84
It: 900, Loss: 1.729e-02, Lambda_1: 0.310, Lambda_2: 0.002886, Time: 2.90
It: 1000, Loss: 1.536e-02, Lambda_1: 0.378, Lambda_2: 0.003043, Time: 2.87
It: 1100, Loss: 1.255e-02, Lambda_1: 0.433, Lambda_2: 0.003233, Time: 3.78
It: 1200, Loss: 1.049e-02, Lambda_1: 0.479, Lambda_2: 0.003369, Time: 3.26
It: 1300, Loss: 9.473e-03, Lambda_1:

In [37]:
np.exp(-0.438876), np.log(0.011676)

(0.6447607250029861, -4.450219825990269)